In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
root_path = 'gdrive/My Drive/Colab Notebooks/sign_language_trans' #change dir to folder where you want to clone
os.chdir('/content/' + root_path)
#!git pull

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install sk-video

In [0]:
import os, sys, time, math, torch, pandas as pd, numpy as np, glob, skvideo.io

from torch import nn, optim
from torch.autograd import Variable
from pathlib import Path

#sys.path.insert(0,"../")


from src.mocogan_model import Discriminator_I, Discriminator_V, Generator_I, GRU

from importlib import reload
import src
reload(src.mocogan_model)

from src.mocogan_model import Discriminator_I, Discriminator_V, Generator_I, GRU

In [0]:
cuda       = 1#cpu
ngpu       = 1
batch_size = 50
n_iter     = 120000
pre_train  = True

In [0]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
if cuda == True:
    torch.cuda.set_device(0)


In [0]:
DATA_DIR = os.path.abspath(os.path.join("data"))

VID_DIR = os.path.join(DATA_DIR, 'raw', "UCF-101")

PROC_VID_DIR = os.path.join(VID_DIR, 'proc_vids')

MODEL_DIR = os.path.join(DATA_DIR, 'models')

OUTPUT_DIR = os.path.join(DATA_DIR, 'outputs')

In [0]:
#resize
def resize_vid(fname, vid_folder):
    
    try:


        from pathlib import Path

        re_dir = os.path.join(os.path.abspath(vid_folder), "proc_vids")

        Path(re_dir).mkdir(parents=True, exist_ok=True)

        proc_fname = re.search("/(.*)$", fname).group(1)

        re_fname = os.path.join(os.path.abspath(re_dir), proc_fname)

        fname_abs = os.path.join(os.path.abspath(vid_folder), fname)

        fname_abs = fname_abs.replace(" ", "\ ")

        re_fname = re_fname.replace(" ", "\ ")

        #print("ffmpeg -i {} -vf scale=98:-2 {}".format(fname_abs, re_fname))
        #os.system("ffmpeg -i {} -vf scale=96:96 {}".format(fname_abs, re_fname))
        !ffmpeg -i {fname_abs} -vf scale=96:96 {re_fname}

    except:
        print("error", fname)
    
    return ""



In [0]:
#!apt install ffmpeg

In [0]:
def read_vid(fname, dir_name):
    
    abs_fname = os.path.join(dir_name,fname)
    
    #print(fname)
    
    return skvideo.io.vread(abs_fname)

In [0]:
"""
classes = os.listdir(VID_DIR)

original_files = [os.path.join(classname,fname) for classname in os.listdir(VID_DIR) for fname in os.listdir(os.path.join(VID_DIR, classname))]

"""

'\nclasses = os.listdir(VID_DIR)\n\noriginal_files = [os.path.join(classname,fname) for classname in os.listdir(VID_DIR) for fname in os.listdir(os.path.join(VID_DIR, classname))]\n\n'

In [0]:
"""
resized_fnames = [resize_vid(x, VID_DIR) for x in original_files[:1000]]

"""

'\nresized_fnames = [resize_vid(x, VID_DIR) for x in original_files[:1000]]\n\n'

In [0]:
files = os.listdir(PROC_VID_DIR)

videos = [ read_vid(file, PROC_VID_DIR) for file in files[:500] ]

# transpose each video to (nc, n_frames, img_size, img_size), and divide by 255
videos = [ video.transpose(3, 0, 1, 2) / 255.0 for video in videos ]

resized_fnames = [resize_vid(x, VID_DIR) for x in files[:1000]]

In [0]:
n_videos = len(videos)

T = 16

In [0]:
# for true video
def trim(video):
    start = np.random.randint(0, video.shape[1] - (T+1))
    end = start + T
    #print(start, end)
    return video[:, start:end, :, :]

def trim_noise(noise):
    start = np.random.randint(0, noise.size(1) - (T+1))
    end = start + T
    return noise[:, start:end, :, :, :]

def random_choice():
    X = []
    for _ in range(batch_size):
        video = videos[np.random.randint(0, n_videos-1)]
        video = torch.Tensor(trim(video))
        X.append(video)
    #print(X.shape)
    X = torch.stack(X)
    return X

In [0]:
# video length distribution
video_lengths = [video.shape[1] for video in videos]

In [0]:
''' set models '''

img_size = 96

nc = 3

ndf = 64 # from dcgan

ngf = 64

d_E = 10

hidden_size = 100 # guess

d_C = 50

d_M = d_E

nz  = d_C + d_M

criterion = nn.BCELoss()

In [0]:
dis_i = Discriminator_I(nc, ndf, ngpu=ngpu)

dis_v = Discriminator_V(nc, ndf, T=T, ngpu=ngpu)

gen_i = Generator_I(nc, ngf, nz, ngpu=ngpu)

gru = GRU(d_E, hidden_size, gpu=cuda)

gru.initWeight()

/content/gdrive/My Drive/Colab Notebooks/sign_language_trans/src/mocogan_model.py:145: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(params)
/content/gdrive/My Drive/Colab Notebooks/sign_language_trans/src/mocogan_model.py:155: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(params, 0)


In [0]:
''' prepare for train '''

label = torch.FloatTensor()

def timeSince(since):
    now = time.time()
    s = now - since
    d = math.floor(s / ((60**2)*24))
    h = math.floor(s / (60**2)) - d*24
    m = math.floor(s / 60) - h*60 - d*24*60
    s = s - m*60 - h*(60**2) - d*24*(60**2)
    return '%dd %dh %dm %ds' % (d, h, m, s)



def checkpoint(model, optimizer, epoch):
    
    filename = os.path.join(MODEL_DIR, '%s_epoch-%d' % (model.__class__.__name__, epoch))
    
    torch.save(model.state_dict(), filename + '.model')
    
    torch.save(optimizer.state_dict(), filename + '.state')

def save_video(fake_video, epoch):
    
    outputdata = fake_video * 255
    
    outputdata = outputdata.astype(np.uint8)
    
    gen_vid_dir = os.path.join(OUTPUT_DIR, 'generated_videos')
    
    Path(gen_vid_dir).mkdir(parents=True, exist_ok=True)
    
    file_path = os.path.join(gen_vid_dir, 'fakeVideo_epoch-%d.mp4' % epoch)
    
    skvideo.io.vwrite(file_path, outputdata)


In [0]:
''' adjust to cuda '''

if cuda == True:
    
    dis_i.cuda()
    
    dis_v.cuda()
    
    gen_i.cuda()
    
    gru.cuda()
    
    criterion.cuda()
    
    label = label.cuda()

In [0]:
# setup optimizer
lr = 0.0002

betas=(0.5, 0.999)

optim_Di  = optim.Adam(dis_i.parameters(), lr=lr, betas=betas)

optim_Dv  = optim.Adam(dis_v.parameters(), lr=lr, betas=betas)

optim_Gi  = optim.Adam(gen_i.parameters(), lr=lr, betas=betas)

optim_GRU = optim.Adam(gru.parameters(),   lr=lr, betas=betas)

In [0]:
pre_train

True

In [0]:
sorted(os.listdir("data/models"))

['Discriminator_I_epoch-1700.model',
 'Discriminator_I_epoch-1700.state',
 'Discriminator_I_epoch-1750.model',
 'Discriminator_I_epoch-1750.state',
 'Discriminator_I_epoch-1800.model',
 'Discriminator_I_epoch-1800.state',
 'Discriminator_I_epoch-1850.model',
 'Discriminator_I_epoch-1850.state',
 'Discriminator_I_epoch-1900.model',
 'Discriminator_I_epoch-1900.state',
 'Discriminator_I_epoch-1950.model',
 'Discriminator_I_epoch-1950.state',
 'Discriminator_I_epoch-2000.model',
 'Discriminator_I_epoch-2000.state',
 'Discriminator_V_epoch-1700.model',
 'Discriminator_V_epoch-1700.state',
 'Discriminator_V_epoch-1750.model',
 'Discriminator_V_epoch-1750.state',
 'Discriminator_V_epoch-1800.model',
 'Discriminator_V_epoch-1800.state',
 'Discriminator_V_epoch-1850.model',
 'Discriminator_V_epoch-1850.state',
 'Discriminator_V_epoch-1900.model',
 'Discriminator_V_epoch-1900.state',
 'Discriminator_V_epoch-1950.model',
 'Discriminator_V_epoch-1950.state',
 'Discriminator_V_epoch-2000.model',
 

In [0]:
''' use pre-trained models '''

if pre_train == True:
    dis_i.load_state_dict(torch.load(MODEL_DIR + '/Discriminator_I_epoch-2000.model'))
    dis_v.load_state_dict(torch.load(MODEL_DIR + '/Discriminator_V_epoch-2000.model'))
    gen_i.load_state_dict(torch.load(MODEL_DIR + '/Generator_I_epoch-2000.model'))
    gru.load_state_dict(torch.load(MODEL_DIR + '/GRU_epoch-2000.model'))
    optim_Di.load_state_dict(torch.load(MODEL_DIR + '/Discriminator_I_epoch-2000.state'))
    optim_Dv.load_state_dict(torch.load(MODEL_DIR + '/Discriminator_V_epoch-2000.state'))
    optim_Gi.load_state_dict(torch.load(MODEL_DIR + '/Generator_I_epoch-2000.state'))
    optim_GRU.load_state_dict(torch.load(MODEL_DIR + '/GRU_epoch-2000.state'))



In [0]:
''' calc grad of models '''

def bp_i(inputs, y, retain=False):
    
    label.resize_(inputs.size(0)).fill_(y)
    
    labelv = Variable(label)
    
    outputs = dis_i(inputs)
    
    err = criterion(outputs, labelv)
    
    err.backward(retain_graph=retain)
    
    return err.data, outputs.data.mean()

def bp_v(inputs, y, retain=False):
    
    label.resize_(inputs.size(0)).fill_(y)
    
    labelv = Variable(label)
    
    outputs = dis_v(inputs)
    
    err = criterion(outputs, labelv)
    
    err.backward(retain_graph=retain)
    
    return err.data, outputs.data.mean()


In [0]:
''' gen input noise for fake video '''

def gen_z(n_frames):
    
    z_C = Variable(torch.randn(batch_size, d_C))
    
    #  repeat z_C to (batch_size, n_frames, d_C)
    
    z_C = z_C.unsqueeze(1).repeat(1, n_frames, 1)
    
    #print(z_C.shape)
    
    eps = Variable(torch.randn(batch_size, d_E))
    
    #print(eps.shape)
    
    if cuda == True:
        z_C, eps = z_C.cuda(), eps.cuda()

    gru.initHidden(batch_size)
    
    # notice that 1st dim of gru outputs is seq_len, 2nd is batch_size
    
    z_M = gru(eps, n_frames).transpose(1, 0)
    
    #print(z_M.shape)
    
    z = torch.cat((z_M, z_C), 2)  # z.size() => (batch_size, n_frames, nz)
    
    #print(z.shape)
    
    return z.view(batch_size, n_frames, nz, 1, 1)


In [0]:
s = trim_noise(gen_z(18))

s.contiguous().view(batch_size*T, nz, 1, 1).shape

torch.Size([800, 60, 1, 1])

In [0]:
''' train models '''

start_time = time.time()

for epoch in range(1975, n_iter+1):
    
    ''' prepare real images '''
    # real_videos.size() => (batch_size, nc, T, img_size, img_size)
    
    real_videos = random_choice()
    
    if cuda == True:
        
        real_videos = real_videos.cuda()
        
    real_videos = Variable(real_videos)
    
    real_img = real_videos[:, :, np.random.randint(0, T), :, :]

    ''' prepare fake images '''
    # note that n_frames is sampled from video length distribution
    
    n_frames = video_lengths[np.random.randint(0, n_videos)]
    
    Z = gen_z(n_frames)  # Z.size() => (batch_size, n_frames, nz, 1, 1)
    
    # trim => (batch_size, T, nz, 1, 1)
    
    Z = trim_noise(Z)
    # generate videos
    
    Z = Z.contiguous().view(batch_size*T, nz, 1, 1)
    # concatenate samples and frames into samples*frame samples of vectors.
    #Each vector will now have a dim of (d_M+d_C)*1*1
    
    fake_videos = gen_i(Z)
    
    fake_videos = fake_videos.view(batch_size, T, nc, img_size, img_size)
    
    # transpose => (batch_size, nc, T, img_size, img_size)
    
    fake_videos = fake_videos.transpose(2, 1)
    
    # img sampling
    fake_img = fake_videos[:, :, np.random.randint(0, T), :, :]

    ''' train discriminators '''
    # video
    dis_v.zero_grad()
    
    err_Dv_real, Dv_real_mean = bp_v(real_videos, 0.9)
    
    err_Dv_fake, Dv_fake_mean = bp_v(fake_videos.detach(), 0)
    
    err_Dv = err_Dv_real + err_Dv_fake
    
    optim_Dv.step()
    
    # image
    dis_i.zero_grad()
    
    err_Di_real, Di_real_mean = bp_i(real_img, 0.9)
    
    err_Di_fake, Di_fake_mean = bp_i(fake_img.detach(), 0)
    
    err_Di = err_Di_real + err_Di_fake
    
    optim_Di.step()


    ''' train generators '''
    
    gen_i.zero_grad()
    
    gru.zero_grad()
    
    # video. notice retain=True for back prop twice
    err_Gv, _ = bp_v(fake_videos, 0.9, retain=True)
    # images
    err_Gi, _ = bp_i(fake_img, 0.9)
    
    optim_Gi.step()
    
    optim_GRU.step()

    if epoch % 5 == 0:
        
        print('[%d / %d] (%s) Loss_Di: %.4f Loss_Dv: %.4f Loss_Gi: %.4f Loss_Gv: %.4f Di_real_mean %.4f Di_fake_mean %.4f Dv_real_mean %.4f Dv_fake_mean %.4f'
              % (epoch, n_iter, timeSince(start_time), err_Di, err_Dv, err_Gi, err_Gv, Di_real_mean, Di_fake_mean, Dv_real_mean, Dv_fake_mean))

    if epoch % 100 == 0:
        
        save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch)

    if epoch % 200 == 0:
        checkpoint(dis_i, optim_Di, epoch)
        checkpoint(dis_v, optim_Dv, epoch)
        checkpoint(gen_i, optim_Gi, epoch)
        checkpoint(gru,   optim_GRU, epoch)

[1975 / 120000] (0d 0h 0m 4s) Loss_Di: 0.3950 Loss_Dv: 0.3335 Loss_Gi: 4.9341 Loss_Gv: 7.8399 Di_real_mean 0.8489 Di_fake_mean 0.0058 Dv_real_mean 0.9018 Dv_fake_mean 0.0005
[1980 / 120000] (0d 0h 0m 26s) Loss_Di: 0.3722 Loss_Dv: 0.3381 Loss_Gi: 4.4868 Loss_Gv: 7.6526 Di_real_mean 0.8906 Di_fake_mean 0.0113 Dv_real_mean 0.8613 Dv_fake_mean 0.0004
[1985 / 120000] (0d 0h 0m 48s) Loss_Di: 0.3439 Loss_Dv: 0.3316 Loss_Gi: 5.4694 Loss_Gv: 8.2683 Di_real_mean 0.9112 Di_fake_mean 0.0051 Dv_real_mean 0.9210 Dv_fake_mean 0.0003
[1990 / 120000] (0d 0h 1m 9s) Loss_Di: 0.3516 Loss_Dv: 0.3283 Loss_Gi: 4.7086 Loss_Gv: 7.7903 Di_real_mean 0.8987 Di_fake_mean 0.0139 Dv_real_mean 0.8973 Dv_fake_mean 0.0003
[1995 / 120000] (0d 0h 1m 31s) Loss_Di: 0.3570 Loss_Dv: 0.3308 Loss_Gi: 4.4630 Loss_Gv: 7.6504 Di_real_mean 0.8870 Di_fake_mean 0.0150 Dv_real_mean 0.9170 Dv_fake_mean 0.0004
[2000 / 120000] (0d 0h 1m 53s) Loss_Di: 0.3790 Loss_Dv: 0.3348 Loss_Gi: 4.3851 Loss_Gv: 6.7330 Di_real_mean 0.8812 Di_fake_mean

In [0]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [0]:
s = trim_noise(gen_z(18))

In [0]:
nz

60